In [1]:
# %%
import pandas as pd
#import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
#import plotly.graph_objects as go
# 출력 제한 해제
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_seq_items', None)
pd.options.display.max_colwidth = 100

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
# %%
train = pd.read_csv('./processed_data/log_train.csv')
test =  pd.read_csv('./processed_data/log_test.csv')
trade = pd.read_csv('./data/international_trade.csv')

train.head()


,index,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),holiday,Date,week,day_name,log_and_filtered_price
0,0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,New Year's Day,2019-01-01,1,Tuesday,0.000000
1,1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,None,2019-01-02,1,Wednesday,0.000000
2,2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,None,2019-01-03,1,Thursday,7.454720
3,3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,None,2019-01-04,1,Friday,7.249926
4,4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,None,2019-01-05,1,Saturday,7.130899


In [3]:
# %%
time = pd.to_datetime(train['timestamp'].copy())

train['Date'] = pd.to_datetime(train['timestamp'])
train['week'] = train['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
train['day_name'] = train['Date'].dt.day_name()

train['year'] = train['timestamp'].apply(lambda x : int(x[0:4]))
train['month'] = train['timestamp'].apply(lambda x : int(x[5:7]))
train['day'] = train['timestamp'].apply(lambda x : int(x[8:10]))

In [4]:
# %%
import datetime
import holidays

# %%
#train = train[(train['supply(kg)'] != 0) & (train['supply(kg)'] != 0)] # 이거차이도 큼!!! 거의 rmse 1200에서 680으로 줄어듦. 근데 이건 validation에만이고 test는 이거하면 정반대로됨 ㅠ...

# %%
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
test_re = test.copy()

test_re['year'] = test_re['timestamp'].apply(lambda x : int(x[0:4]))
test_re['month'] = test_re['timestamp'].apply(lambda x : int(x[5:7]))
test_re['day'] = test_re['timestamp'].apply(lambda x : int(x[8:10]))
test_re['Date'] = pd.to_datetime(test_re['timestamp'])
test_re['week'] = test_re['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
test_re['day_name'] = test_re['Date'].dt.day_name()


# %%
test_re.head()

,ID,timestamp,item,corporation,location,holiday,Date,week,day_name,year,month,day
0,TG_A_J_20230304,2023-03-04,TG,A,J,None,2023-03-04,9,Saturday,2023,3,4
1,TG_A_J_20230305,2023-03-05,TG,A,J,None,2023-03-05,9,Sunday,2023,3,5
2,TG_A_J_20230306,2023-03-06,TG,A,J,None,2023-03-06,10,Monday,2023,3,6
3,TG_A_J_20230307,2023-03-07,TG,A,J,None,2023-03-07,10,Tuesday,2023,3,7
4,TG_A_J_20230308,2023-03-08,TG,A,J,None,2023-03-08,10,Wednesday,2023,3,8


In [17]:
# %%
from sklearn.preprocessing import LabelEncoder
train_x = train.drop(columns=['ID', 'timestamp', 'supply(kg)', 'price(원/kg)','Date'])
train_y = train['price(원/kg)']
test_x = test_re.drop(columns=['ID', 'timestamp','Date'])
qual_col = ['corporation', 'location','day_name']


for i in ['holiday']:
    le = LabelEncoder()
    train_x[i]=le.fit_transform(train_x[i])
    test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다

# train_x = pd.get_dummies(train_x, columns=qual_col, prefix=qual_col)
# test_x = pd.get_dummies(test_x, columns=qual_col, prefix=qual_col)

In [18]:
display(train_x.head(5))
display(test_x.head(5))

,index,item,corporation,location,holiday,week,day_name,log_and_filtered_price,year,month,day
0,0,TG,A,J,19,1,Tuesday,0.000000,2019,1,1
1,1,TG,A,J,20,1,Wednesday,0.000000,2019,1,2
2,2,TG,A,J,20,1,Thursday,7.454720,2019,1,3
3,3,TG,A,J,20,1,Friday,7.249926,2019,1,4
4,4,TG,A,J,20,1,Saturday,7.130899,2019,1,5


,item,corporation,location,holiday,week,day_name,year,month,day
0,TG,A,J,20,9,Saturday,2023,3,4
1,TG,A,J,20,9,Sunday,2023,3,5
2,TG,A,J,20,10,Monday,2023,3,6
3,TG,A,J,20,10,Tuesday,2023,3,7
4,TG,A,J,20,10,Wednesday,2023,3,8


In [41]:
# import pycaret regression and init setup
from pycaret.regression import *

predicts = []

for item in train_x.item.unique():
    print(item)
    item_train_x = train_x[train_x['item']==item]
    item_test_x = test_x[test_x['item']==item]
    print(item_test_x.shape)
    item_train_x = item_train_x.drop(columns=['item'])
    item_test_x = item_test_x.drop(columns=['item'])

    item_train_x = item_train_x.drop(columns = ['index'])

    item_train_x = pd.get_dummies(item_train_x, columns=qual_col, prefix=qual_col)
    item_test_x = pd.get_dummies(item_test_x, columns=qual_col, prefix=qual_col)
    
    s = setup(item_train_x, target = 'log_and_filtered_price', 
            session_id = 123, 
            fold = 10, 
            normalize = True, 
            normalize_method = 'minmax',
            train_size=0.9)
    
    # Best models 선택 (예: 상위 3개 모델 선택)
    reg = create_model('rf')
    
    tuned_reg = tune_model(reg, optimize = 'RMSE', fold=10, n_iter=20)
    predictions = predict_model(tuned_reg, data = pd.DataFrame(item_test_x))
    predicts.extend(list(np.exp(predictions['prediction_label'])))

TG
(280, 9)


,Description,Value
0,Session id,123
1,Target,log_and_filtered_price
2,Target type,Regression
3,Original data shape,"(13041, 19)"
4,Transformed data shape,"(13041, 19)"
5,Transformed train set shape,"(11736, 19)"
6,Transformed test set shape,"(1305, 19)"
7,Numeric features,18
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.0182,4.3128,2.0767,0.3656,0.5549,0.0780
1,0.9957,4.3404,2.0834,0.3607,0.5548,0.0733
2,0.9543,3.8783,1.9693,0.4106,0.5401,0.0710
3,1.0756,4.5503,2.1331,0.3308,0.5737,0.0792
4,0.9912,4.0743,2.0185,0.4174,0.5555,0.0711
5,1.0160,4.5763,2.1392,0.3839,0.5857,0.0685
6,1.0039,4.1857,2.0459,0.3553,0.5471,0.0759
7,0.9385,3.5958,1.8962,0.4431,0.5278,0.0692
8,0.9397,3.8931,1.9731,0.3938,0.5388,0.0683


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.2398,4.5421,2.1312,0.3318,0.6198,0.0870
1,1.2210,4.6138,2.1480,0.3204,0.6238,0.0831
2,1.1847,4.2778,2.0683,0.3499,0.6059,0.0827
3,1.2544,4.7710,2.1843,0.2983,0.6342,0.0849
4,1.2474,4.7331,2.1756,0.3232,0.6342,0.0843
5,1.2654,5.0694,2.2515,0.3175,0.6614,0.0791
6,1.2192,4.4064,2.0991,0.3213,0.6061,0.0865
7,1.1881,4.4121,2.1005,0.3166,0.6105,0.0811
8,1.1844,4.4156,2.1013,0.3124,0.6104,0.0813


Fitting 10 folds for each of 20 candidates, totalling 200 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


...

BC
(252, 9)


,Description,Value
0,Session id,123
1,Target,log_and_filtered_price
2,Target type,Regression
3,Original data shape,"(9126, 19)"
4,Transformed data shape,"(9126, 19)"
5,Transformed train set shape,"(8213, 19)"
6,Transformed test set shape,"(913, 19)"
7,Numeric features,18
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4947,1.8499,1.3601,0.8804,0.3941,0.0709
1,0.5495,2.1419,1.4635,0.8620,0.4337,0.0724
2,0.5503,2.1120,1.4533,0.8632,0.4343,0.0719
3,0.5578,2.2863,1.5120,0.8519,0.4088,0.0913
4,0.5481,2.1839,1.4778,0.8577,0.4512,0.0722
5,0.5735,2.2269,1.4923,0.8555,0.4334,0.0914
6,0.6777,2.6894,1.6399,0.8273,0.4646,0.1017
7,0.5821,2.5614,1.6004,0.8341,0.4816,0.0693
8,0.4911,1.9721,1.4043,0.8726,0.3970,0.0739


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.8827,2.3141,1.5212,0.8504,0.4883,0.1177
1,0.9282,2.5242,1.5888,0.8374,0.5184,0.1168
2,0.8933,2.3763,1.5415,0.8461,0.5032,0.1136
3,0.8709,2.3693,1.5393,0.8465,0.4689,0.1222
4,0.9607,3.1194,1.7662,0.7968,0.5515,0.1230
5,1.0118,2.9851,1.7277,0.8063,0.5416,0.1411
6,1.0552,3.0611,1.7496,0.8034,0.5304,0.1521
7,0.9359,2.9472,1.7168,0.8091,0.5541,0.1097
8,0.8791,2.4647,1.5699,0.8408,0.4900,0.1204


Fitting 10 folds for each of 20 candidates, totalling 200 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [42]:
len(predicts)

1092

In [43]:
# final_model = finalize_model(blended)

In [44]:
result = test_re.copy()
result['answer'] = predicts

del result['timestamp']
del result['item']
del result['corporation']
del result['location']
del result['year']
del result['month']
del result['day']
del result['Date']
del result['week']
del result['day_name']
del result['holiday']


In [45]:
for i in range(len(result)):
    if (i-1) % 7 == 0: # 1, 8, 15, 22
        result.iloc[i,-1] = 0

result.head(20)

,ID,answer
0,TG_A_J_20230304,1705.414556
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,1755.872710
3,TG_A_J_20230307,2976.816413
4,TG_A_J_20230308,2979.262105
5,TG_A_J_20230309,2767.511015
6,TG_A_J_20230310,2570.168100
7,TG_A_J_20230311,1930.667856
8,TG_A_J_20230312,0.000000
9,TG_A_J_20230313,3087.503950


In [46]:
result.to_csv('result_hard_tuened.csv', index = False)

In [47]:
result

,ID,answer
0,TG_A_J_20230304,1705.414556
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,1755.872710
3,TG_A_J_20230307,2976.816413
4,TG_A_J_20230308,2979.262105
...,...,...
1087,RD_F_J_20230327,2684.202266
1088,RD_F_J_20230328,281.151479
1089,RD_F_J_20230329,610.587714
1090,RD_F_J_20230330,757.853311


In [ ]:
train_x.to_csv('./processed_data/train.csv')
test_x.to_csv('./processed_data/test.csv')